# Module 02: State Management - Follow Along

**Purpose:** Run state management examples from theory  
**After this:** Move to `module-02-practice.ipynb`

---

Master state schemas, reducers, and advanced patterns.


In [ ]:
%pip install -q -U langgraph langchain

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END, add_messages

print('✅ Ready!')

## Example 1: State Schema Patterns


In [ ]:
# Flat schema
class FlatState(TypedDict):
    user_input: str
    llm_response: str
    confidence: float

# Nested schema
class NestedState(TypedDict):
    user: dict
    session: dict
    conversation: dict

# Hybrid schema (recommended)
class HybridState(TypedDict):
    messages: Annotated[list, add_messages]  # Flat, frequently accessed
    current_status: str
    user_profile: dict  # Nested for logical grouping
    metadata: dict

print("Schema types defined - Hybrid is recommended!")

## Example 2: Built-in Reducers


In [ ]:
# Default reducer (replace)
class DefaultState(TypedDict):
    count: int  # No reducer annotation

# add_messages reducer  
class MessageState(TypedDict):
    messages: Annotated[list, add_messages]

# Test default reducer
def node1(state: DefaultState):
    return {'count': 10}

def node2(state: DefaultState):
    return {'count': 20}  # Replaces 10

workflow = StateGraph(DefaultState)
workflow.add_node('n1', node1)
workflow.add_node('n2', node2)
workflow.add_edge(START, 'n1')
workflow.add_edge('n1', 'n2')
workflow.add_edge('n2', END)

app = workflow.compile()
result = app.invoke({'count': 5})
print(f"Default reducer (replace): {result['count']}")  # Will be 20

## Example 3: Custom Reducers - All Patterns


In [ ]:
# Pattern 1: Accumulator
def accumulate_list(e: list, n: list) -> list:
    return e + n

# Pattern 2: Max selector
def keep_max(e: float, n: float) -> float:
    return max(e, n)

# Pattern 3: Dict merger
def merge_dicts(e: dict, n: dict) -> dict:
    result = e.copy()
    result.update(n)
    return result

# Pattern 4: Deduplicator
def dedupe_items(e: list, n: list) -> list:
    e_set = set(e)
    return e + [item for item in n if item not in e_set]

# Pattern 5: Counter
def increment_counter(e: int, n: int) -> int:
    return e + n

class AllReducersState(TypedDict):
    logs: Annotated[list, accumulate_list]
    max_score: Annotated[float, keep_max]
    config: Annotated[dict, merge_dicts]
    tags: Annotated[list, dedupe_items]
    iterations: Annotated[int, increment_counter]

print("All 5 custom reducer patterns defined!")

## Example 4: Graph Execution Models


In [ ]:
# Sequential execution
class SeqState(TypedDict):
    value: int

workflow = StateGraph(SeqState)
workflow.add_node('n1', lambda s: {'value': s['value'] + 1})
workflow.add_node('n2', lambda s: {'value': s['value'] * 2})
workflow.add_node('n3', lambda s: {'value': s['value'] - 3})
workflow.add_edge(START, 'n1')
workflow.add_edge('n1', 'n2')
workflow.add_edge('n2', 'n3')
workflow.add_edge('n3', END)

app = workflow.compile()
result = app.invoke({'value': 5})
print(f"Sequential: (5 + 1) * 2 - 3 = {result['value']}")

## Example 5: State Flow Patterns


In [ ]:
# Pipeline pattern
class PipelineState(TypedDict):
    raw_data: str
    cleaned_data: str
    processed_data: str
    result: str

def clean(s: PipelineState):
    return {'cleaned_data': s['raw_data'].strip().lower()}

def process(s: PipelineState):
    return {'processed_data': f"Processed: {s['cleaned_data']}"}

def analyze(s: PipelineState):
    return {'result': f"Analysis of {s['processed_data']}"}

workflow = StateGraph(PipelineState)
workflow.add_node('clean', clean)
workflow.add_node('process', process)
workflow.add_node('analyze', analyze)
workflow.add_edge(START, 'clean')
workflow.add_edge('clean', 'process')
workflow.add_edge('process', 'analyze')
workflow.add_edge('analyze', END)

app = workflow.compile()
result = app.invoke({
    'raw_data': '  HELLO WORLD  ',
    'cleaned_data': '', 'processed_data': '', 'result': ''
})
print(f"Pipeline result: {result['result']}")

## Example 6: State Validation


In [ ]:
class ValidState(TypedDict):
    user_id: str
    email: str
    errors: list
    is_valid: bool

def validate(state: ValidState):
    errors = []
    if not state.get('user_id'):
        errors.append('Missing user_id')
    if not state.get('email') or '@' not in state.get('email', ''):
        errors.append('Invalid email')
    return {'errors': errors, 'is_valid': len(errors) == 0}

# Test
workflow = StateGraph(ValidState)
workflow.add_node('validate', validate)
workflow.add_edge(START, 'validate')
workflow.add_edge('validate', END)

app = workflow.compile()

# Valid
r1 = app.invoke({'user_id': 'u1', 'email': 'test@example.com', 'errors': [], 'is_valid': False})
print(f"Valid: {r1['is_valid']}, Errors: {r1['errors']}")

# Invalid
r2 = app.invoke({'user_id': '', 'email': 'bad', 'errors': [], 'is_valid': False})
print(f"Invalid: {r2['is_valid']}, Errors: {r2['errors']}")

## Summary

You've run examples for:
- ✅ State schema patterns (flat, nested, hybrid)
- ✅ Built-in reducers (default, add_messages)
- ✅ 5 custom reducer patterns
- ✅ Graph execution models
- ✅ State flow patterns (pipeline, aggregation)
- ✅ State validation techniques

**Next:** `module-02-practice.ipynb` for hands-on exercises!
